In [1]:
system("sudo apt install python3-venv python3-pip python3-dev")

In [5]:
# Install and load required packages
install.packages(c("keras", "tensorflow", "ggplot2", "caret"))
library(keras)
library(tensorflow)
library(ggplot2)
library(caret)
library(readxl)
# Install TensorFlow (run once)
install_tensorflow()

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Virtual environment 'r-tensorflow' removed.
Using Python: /usr/bin/python3.10
Creating virtual environment 'r-tensorflow' ... 


+ /usr/bin/python3.10 -m venv /root/.virtualenvs/r-tensorflow



Done!
Installing packages: pip, wheel, setuptools


+ /root/.virtualenvs/r-tensorflow/bin/python -m pip install --upgrade pip wheel setuptools



Virtual environment 'r-tensorflow' successfully created.
Using virtual environment 'r-tensorflow' ...


+ /root/.virtualenvs/r-tensorflow/bin/python -m pip install --upgrade --no-user 'tensorflow==2.16.*'




Installation complete.



In [6]:
# Load your data
df <- read_excel('final.xlsx')
df <- df[2:17]  # Select relevant columns

In [7]:

# Convert target to factor and numeric
df$mq_type <- as.factor(df$mq_type)
y <- as.numeric(df$mq_type) - 1  # Convert to 0,1,2 for deep_mq, impact_mq, shallow_mq

In [8]:
# Remove constant columns (like Arrival Time)
constant_cols <- sapply(df, function(x) length(unique(x)) == 1)
df <- df[, !constant_cols]

In [9]:
# Split data into training and test sets
set.seed(123)
train_index <- createDataPartition(y, p = 0.8, list = FALSE)
train_data <- df[train_index, ]
test_data <- df[-train_index, ]

In [10]:
# Scale numeric features (excluding the target variable)
preprocess_params <- preProcess(train_data[, -which(names(train_data) == "mq_type")],
                              method = c("center", "scale"))
train_scaled <- predict(preprocess_params, train_data[, -which(names(train_data) == "mq_type")])
test_scaled <- predict(preprocess_params, test_data[, -which(names(test_data) == "mq_type")])


In [12]:
# Convert to matrices for keras
x_train <- as.matrix(train_scaled)
x_test <- as.matrix(test_scaled)


In [14]:
library(caret)
dummy <- dummyVars(" ~ .", data = data.frame(mq_type = train_data$mq_type))
y_train <- predict(dummy, newdata = data.frame(mq_type = train_data$mq_type))
y_test <- predict(dummy, newdata = data.frame(mq_type = test_data$mq_type))


In [21]:
colnames(df)

[1] "Mean Velocity"                "Standard Deviation"          
 [3] "Max Velocity"                 "Min Velocity"                
 [5] "Range Velocity"               "Median Velocity"             
 [7] "RMS Velocity"                 "Energy"                      
 [9] "Impulse Factor"               "Velocity Derivative Mean"    
[11] "Velocity Derivative Std"      "Mean Velocity Around Arrival"
[13] "Max Velocity Around Arrival"  "Max STA/LTA"                 
[15] "mq_type"

In [23]:
colnames(df)
colnames(df) <- gsub(" ", "_", colnames(df))
colnames(df) <- gsub("/", "_", colnames(df))
colnames(df)

[1] "Mean Velocity"                "Standard Deviation"          
 [3] "Max Velocity"                 "Min Velocity"                
 [5] "Range Velocity"               "Median Velocity"             
 [7] "RMS Velocity"                 "Energy"                      
 [9] "Impulse Factor"               "Velocity Derivative Mean"    
[11] "Velocity Derivative Std"      "Mean Velocity Around Arrival"
[13] "Max Velocity Around Arrival"  "Max STA/LTA"                 
[15] "mq_type"

[1] "Mean_Velocity"                "Standard_Deviation"          
 [3] "Max_Velocity"                 "Min_Velocity"                
 [5] "Range_Velocity"               "Median_Velocity"             
 [7] "RMS_Velocity"                 "Energy"                      
 [9] "Impulse_Factor"               "Velocity_Derivative_Mean"    
[11] "Velocity_Derivative_Std"      "Mean_Velocity_Around_Arrival"
[13] "Max_Velocity_Around_Arrival"  "Max_STA_LTA"                 
[15] "mq_type"

In [25]:
# Install if not already installed
install.packages("randomForest")
library(randomForest)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [26]:
# Prepare data
df$mq_type <- as.factor(df$mq_type)  # Make sure target is a factor
df <- df[, !sapply(df, function(x) length(unique(x)) == 1)]  # Remove constant columns

In [27]:
# Split into train and test
set.seed(123)
train_idx <- sample(1:nrow(df), 0.8 * nrow(df))
train_data <- df[train_idx, ]
test_data <- df[-train_idx, ]

In [28]:
# Train a simple Random Forest
model_rf <- randomForest(mq_type ~ ., data = train_data, ntree = 100)

In [29]:
# Predict
pred <- predict(model_rf, newdata = test_data)

In [30]:
# Confusion matrix
confusion <- table(Predicted = pred, Actual = test_data$mq_type)
print(confusion)

            Actual
Predicted    deep_mq impact_mq shallow_mq
  deep_mq          0         0          0
  impact_mq        1        14          0
  shallow_mq       0         0          0


In [31]:

# Accuracy
accuracy <- sum(diag(confusion)) / sum(confusion)
cat("Accuracy:", accuracy, "\n")

Accuracy: 0.9333333 
